In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-10-12 11:56:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3,7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3,74K  --.-KB/s    in 0s      

2024-10-12 11:56:49 (78,3 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
# with open('documents.json', 'rt') as f_in:
#     docs_raw = json.load(f_in)
with open('translated_luat_giao_duc.txt', 'r', encoding='utf-8') as f_in:
    law_text = f_in.read()

In [5]:
# documents = []

# for course_dict in docs_raw:
#     for doc in course_dict['documents']:
#         doc['course'] = course_dict['course']
#         documents.append(doc)
documents = []

# Assuming you want each section to be a 'document' in the dataset
documents = [{"text": law_text}]

# Example of the first document created
print(documents[0])

{'text': "National Assembly\n-------\n\nSocialist Republic of Vietnam\nIndependence - Freedom - Happiness\n---------------\n\nLaw No. 43/2019/QH14\n\nHanoi, June 14, 2019\n\n\n\nLAW\n\nEDUCATION\n\nPursuant to the Constitution of the Socialist Republic of Vietnam;\n\nThe National Assembly promulgates the Education Law.\n\nChapter I\n\nGeneral regulations\n\nArticle 1. Scope of adjustment\n\nThis law stipulates the national education system;educational institutions, teachers, learners;State management of education;rights and responsibilities of agencies, organizations and individuals related to educational activities.\n\nArticle 2. Education goals\n\nEducation goals to comprehensively develop Vietnamese people with morality, knowledge, culture, health, aesthetics and occupation;have citizen quality, capacity and consciousness;have patriotism, national spirit, loyalty to the ideal of national independence and socialism;promote the potential and creative ability of each individual;Improve

In [7]:
documents[1]

IndexError: list index out of range

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [10]:
q = 'What are the educational goals outlined for developing Vietnamese citizens in terms of morality, knowledge, and national spirit, as per the law?'

In [11]:
index.fit(documents)

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:
query = 'how do I run kafka?'
search_results = search(query)

search_results

[{'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming with kafka',
  'question': 'Module “kafka” not found when trying to run producer.py',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
prompt = build_prompt(query, search_results)

In [16]:
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n    \n    QUESTION: how do I run kafka?\n    \n    CONTEXT: \n    section: Module 6: streaming with kafka\nquestion: Module “kafka” not found when trying to run producer.py\nanswer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you\'ll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it\'s env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.\n\nse

In [17]:
# from openai import OpenAI
# client = OpenAI()
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role": "user", "content": q}]
# )

# response.choices[0].message.content

# def llm(prompt):
#     response = client.chat.completions.create(
#         model='gpt-4o',
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content

In [20]:
! source ~/.baschrc

In [22]:
import os

api_key = os.environ["MISTRAL_API_KEY"]


In [27]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

client = MistralClient(api_key=api_key)


def llm(prompt):
    response = client.chat(
        model='open-mistral-7b',
        messages=[
            ChatMessage(role="user", content=prompt)
        ]
    )

    return response.choices[0].message.content

In [29]:
response = llm(prompt)

In [30]:
response

'To run Kafka, follow these steps:\n\n1. First, create a virtual environment and install the required packages:\n   - On MacOS, Linux, and Linux (for Windows, the path is slightly different):\n     ```\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n   ```\n   - To activate it, run:\n     ```\n     source env/bin/activate\n     ```\n   - To deactivate it, run:\n     ```\n     deactivate\n     ```\n\n2. Ensure that the \'dlt[duckdb]\' package is installed:\n   - Execute the provided installation command:\n     ```\n     !pip install dlt[duckdb]\n     ```\n\n3. If you have a permission denied error when running `./build.sh`, run this command in the terminal:\n   ```\n   chmod +x build.sh\n   ```\n\n4. If you encounter an error "ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'", use the following command to install a different version of Kafka Python:\n   ```\n   pip install kafka-python-ng\n   ```\n\nNow, you should be able 

In [31]:
query = 'the course has already started, can I still enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
rag(query)

'Based on the provided context, you can still enroll in the course even if it has already started. However, you should be aware that there will be deadlines for turning in the final projects. The course will start on January 15, 2024, at 17h00. Before the course starts, you can prepare by installing and setting up all the dependencies and requirements such as a Google cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, and Git. You can also look over the prerequisites and syllabus to see if you are comfortable with these subjects. If you take the course in the self-paced mode, you can still get support by asking questions in the Slack channel.'

In [20]:
from elasticsearch import Elasticsearch

In [21]:
es_client = Elasticsearch('http://localhost:9200') 

In [23]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████| 948/948 [00:28<00:00, 33.07it/s]


In [36]:
query = 'I just disovered the course. Can I still join it?'

In [42]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [44]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
rag(query)

'Yes, you can still join the course even if you discovered it after the start date. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects. So make sure not to leave everything for the last minute.'